![Logo TUBAF](https://tu-freiberg.de/sites/default/files/media/freiberger-alumni-netzwerk-6127/wbm_orig_rgb_0.jpg)

Exercise material of the MSc-level course **Soil Dynamics**.
Held at Technische Universität Bergakademie Freiberg.

Comments to:

*Prof. Dr. Thomas Nagel  
Chair of Soil Mechanics and Foundation Engineering  
Geotechnical Institute  
Technische Universität Bergakademie Freiberg.*

https://tu-freiberg.de/en/soilmechanics


# Einmassenschwinger, Grundlagen

![oscillator1dof.png](https://github.com/nagelt/soil_dynamics/raw/master/Images/oscillator1dof.png)

## Hilfsfunktionen

Module laden

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets
from ipywidgets import interact
import plot_functions.plot_settings
%run plot_functions/single_mass_oscillator_interaction.ipynb

Umwandlung sin-cos-Darstellung in Amplitude-Phasenwinkel und zurück (nur zur Info und Überprüfung, wird nicht zur Lösung gebraucht)

In [2]:
def sc2ap(s, c):
    a = np.sqrt(s**2 + c**2)
    p = np.arctan2(s, c)
    return [a, p]

In [3]:
def ap2sc(a, p):
    s = a*np.sin(p)
    c = a*np.cos(p)
    return [s, c]

Umrechnung der physikalischen Parameter in die Schwingungsparameter (Standardform)

In [4]:
def standard_form(m, c, k, F_c, F_s):
    if k==0 or m==0:
        print("Warning: unphysical parameters for an oscillator!")
    omega_0 = np.sqrt(k/m)
    zeta = c/(2*np.sqrt(k*m))
    a_c = F_c/k
    a_s = F_s/k
    return [omega_0, zeta, a_c, a_s]

Eingeschwungenen Zustand (partikuläre Lösung) berechnen, inklusive Vergrößerungsfunktion und Phasendifferenz

In [5]:
def steady_state_solution(omega, omega_0, zeta, a_c, a_s):
    eta = omega/omega_0
    if eta==1 and zeta==0:   # TODO implement solutions up1(t) and up2(t) for this case as done for uh(t)
        print("Warning: no steady-state solution for resonant excitation of undamped oscillator!")
        V = 0 
        psi = 0
        u_c = 0
        u_s = 0
    else:        
        V2 = 1.0/( (1-eta**2)**2 + (2*eta*zeta)**2 )
        V = np.sqrt(V2)
        psi = np.arctan2(2*zeta*eta, 1-eta**2)
        u_c = V2*( (1-eta**2)*a_c - 2*zeta*eta*a_s )
        u_s = V2*( (1-eta**2)*a_s + 2*zeta*eta*a_c )
    return [u_c, u_s, V, psi]

Anpassen der Gesamtlösung an die Anfangsbedingungen

In [6]:
def transient_solution(t_0, u_0, v_0):
    uh_c1 = uh_1(t_0)
    uh_c2 = uh_2(t_0)
    vh_c1 = vh_1(t_0)
    vh_c2 = vh_2(t_0)
    up_0 = up(t_0)
    vp_0 = vp(t_0)
    # LGS Ax=b
    A = np.array([[uh_c1, uh_c2], [vh_c1, vh_c2]])
    b = np.array([u_0-up_0, v_0-vp_0])
    x = np.linalg.solve(A, b)
    C_1 = x[0]
    C_2 = x[1]
    return [C_1, C_2]

Berechnen des Logarithmischen Dekrements (wichtige Größe in der experimentellen Schwingungsmessung) 

In [7]:
def log_decrement(t_0, t_N, u_0, u_N, N):
    LD = (1.0/N)*np.log(u_0/u_N)
    return LD

## Simulation 

#### Passen Sie die Parameter entsprechend an:

Freie ($F_c=F_s=0$), ungedämpfte ($c=0$) oder gedämpfte ($c>0$) Schwingung

Erzwungene ($\hat{F}>0$), ungedämpfte ($c=0$) oder gedämpfte ($c>0$) Schwingung

In [8]:
m = 3.0   # kg
k = 10.0   # N/m
c = 1.0   # Ns/m   critical damping for c=2*sqrt(k*m)=10.95445115...
F_c = 0.0   # N
F_s = 5.0   # N
omega = 10.0   # rad/s
t_0 = 0.0   # s
u_0 = 0.1   # m
v_0 = 0.0   # m/s

Berechnen der Schwingungsparameter und der zur Kraftanregung äquivalenten Federfusspunktanregung

In [9]:
[omega_0, zeta, a_c, a_s] = standard_form(m, c, k, F_c, F_s)   
print("omega_0={:.3f} rad/s".format(omega_0), "   zeta={:.3f}".format(zeta))

# define harmonic excitation as function a(t)
a = lambda t: a_c*np.cos(omega*t) + a_s*np.sin(omega*t)

omega_0=1.826 rad/s    zeta=0.091


Berechnen des eingeschwungenen Zustands (partikuläre Lösung)

In [10]:
[u_c, u_s, V, psi] = steady_state_solution(omega, omega_0, zeta, a_c, a_s)
print("V={:.3f}".format(V), "   psi={:.3f} rad".format(psi))
print("u_c={:.4f} m".format(u_c), "   u_s={:.4f} m".format(u_s))

# define steady state solution u_p(t) and its time derivative v_p(t)
up = lambda t: u_c*np.cos(omega*t) + u_s*np.sin(omega*t)
vp = lambda t: (-u_c*np.sin(omega*t) + u_s*np.cos(omega*t)) * omega

V=0.034    psi=3.107 rad
u_c=-0.0006 m    u_s=-0.0172 m


Anpassen der Gesamtlösung an Anfangsbedingungen, je Dämpfungsgrad (über-/unter-/kritisch) 

In [12]:
# define two solutions u_h(t) and their time derivative v_h(t), depending on discriminant
if zeta < 1:
    omega_1 = omega_0*np.sqrt(1-zeta**2)
    delta = omega_0*zeta
    print("omega_1={:.3f} rad/s".format(omega_1), "   delta={:.3f} 1/s".format(delta))
    uh_1 = lambda t: np.exp(-delta*t)*np.cos(omega_1*t)
    uh_2 = lambda t: np.exp(-delta*t)*np.sin(omega_1*t)
    vh_1 = lambda t: np.exp(-delta*t)*( -delta*np.cos(omega_1*t) - omega_1*np.sin(omega_1*t) )
    vh_2 = lambda t: np.exp(-delta*t)*( -delta*np.sin(omega_1*t) + omega_1*np.cos(omega_1*t) )
elif zeta > 1:
    delta_1 = omega_0*(zeta - np.sqrt(zeta**2 - 1))
    delta_2 = omega_0*(zeta + np.sqrt(zeta**2 - 1))
    print("delta_1={:.3f} 1/s".format(delta_1), "   delta_2={:.3f} 1/s".format(delta_2) )
    uh_1 = lambda t: np.exp(-delta_1*t)
    uh_2 = lambda t: np.exp(-delta_2*t)
    vh_1 = lambda t: np.exp(-delta_1*t)*(-delta_1)
    vh_2 = lambda t: np.exp(-delta_2*t)*(-delta_2)
else:
    delta = omega_0*zeta
    print("delta={:.3f} 1/s".format(delta), "   (critical damping)")
    uh_1 = lambda t: np.exp(-delta*t)
    uh_2 = lambda t: t*np.exp(-delta*t)
    vh_1 = lambda t:-np.exp(-delta*t)*delta
    vh_2 = lambda t: np.exp(-delta*t) - t*np.exp(-delta*t)*delta
    
[C_1, C_2] = transient_solution(t_0, u_0, v_0)
print("C_1={:.3f} m".format(C_1), "C_2={:.3f} m".format(C_2))

# define adjusted solution of homogenous solution and total solution
uh = lambda t: C_1*uh_1(t) + C_2*uh_2(t) 
u = lambda t:  uh(t) + up(t) 

omega_1=1.818 rad/s    delta=0.167 1/s
C_1=0.101 m C_2=0.104 m


Erstellen diskreter Lösungsverläufe auf einem Zeitraster und grafische Darstellung

In [ ]:
Nplot=5
t_5P = t_0 + Nplot*2*np.pi/omega_0   # <Nplot> periods of free vibrations (undamped)
t_sample = np.linspace(t_0, t_5P , 250)   # discretization with 250 points in time
a_sample = a(t_sample)
uh_sample = uh(t_sample)
up_sample = up(t_sample)
u_sample = u(t_sample)

In [ ]:
plt.plot(t_sample, uh_sample, 'b--', t_sample, up_sample, 'b:', t_sample, u_sample, 'b');
plt.xlabel("$t$");
plt.ylabel("$u$");

In [ ]:
plt.plot(t_sample, a_sample, 'r', t_sample, up_sample, 'b:');
plt.xlabel("$t$");
plt.ylabel("$u, a$");

**Diskussion:** Wie würden Sie das Einschalten einer Maschine simulieren, d.h. mit welchen Parametern und Anfangsbedingungen?

In [ ]:
interactive_tau_u_plot()   # nondimensional parameters

## Parameterbestimmung

### Ausschwingversuch

In [ ]:
t1 = 0   # s
t2 = 1.0   # s
u1 = 0.1   # m
u2 = 0.08   # m
N = 1

Was wir im Experiment sehen ist die gedämpfte Eigenfrequenz

In [ ]:
T_id = t2 - t1
omega_1_id = 2.0*np.pi/T_id

Nach einer Periode, wiederholen sich die harmonischen Funktionen, dadurch erfassen wir das Abklingen $e^{-\delta t}$ und nutzen $e^{-\delta t_2}=e^{-\delta(t_1+T)}=e^{-\delta t_1} e^{-\delta T}$

In [ ]:
LD = log_decrement(t1, t2, u1, u2, N)
delta_id = LD/T_id
c_id = 2*delta_id*m
print("c_id={:.3f} Ns/m".format(c_id) )

Wir erinnern uns an $\omega_1=\omega_0\sqrt{1-\zeta^2}$ und $\delta=\zeta \omega_0$

In [ ]:
omega_0_id_squared = omega_1_id**2 + delta_id**2
k_id = omega_0_id_squared*m
print("k_id={:.3f} N/m".format(k_id))

### Schwingsaitenwaage

In [ ]:
omega_measured = 10.0
zeta_measured = 0.1

ungedämpfte Eigenkreisfrequenz $\omega_0=\sqrt{\frac{k}{m}}$

In [ ]:
omega_0_measured_1 = omega_measured
k_measured_1 = m*omega_0_measured_1**2    
print("k1={:.3f} N/m".format(k_measured_1) )

gedämpfte Eigenkreisfrequenz $\omega_1=\omega_0\sqrt{1-\zeta^2}$

In [ ]:
omega_0_measured_2 = omega_measured/np.sqrt(1-zeta_measured**2)
k_measured_2 = m*omega_0_measured_2**2    
print("k2={:.3f} N/m".format(k_measured_2), "   Delta={:2.2f} %".format(100*np.abs(k_measured_2-k_measured_1)/k_measured_1))

maximale Antwortamplitude bei der Anregungsfrequenz $\omega_\mathrm{MA}=\omega_0\sqrt{1-2\zeta^2}$

In [ ]:
omega_0_measured_3 = omega_measured/np.sqrt(1-2*zeta_measured**2)
k_measured_3 = m*omega_0_measured_3**2    
print("k3={:.3f} N/m".format(k_measured_3), "   Delta={:2.2f} %".format(100*np.abs(k_measured_3-k_measured_1)/k_measured_1))

**TODO** Energiebilanz über einen Zyklus Ekin, Epot, Etotal